In [56]:
import pandas as pd
import numpy as np
import plotly.express as px

In [57]:
final_merged = pd.read_csv('final_combined_df.csv', index_col=0)
employment=pd.read_csv('ca_counties_employment_2020.csv',index_col=0)
employment.area_name = employment.area_name.apply(lambda x: x.split('county')[0])
employment.rename(columns={'area_name':'county'}, inplace=True)
employment['month'] = employment.date.apply(lambda x: x.split('-')[1])
employment['county']=employment['county'].str.strip(' ')
final_merged.head(2)

,ppp_loan_amount,business_name,address,city,state,zipcode,naics_code,business_type,jobs_reported,date_approved,...,zipcode_type,county,county_total_loan_amount,q1,q2,q_dif,date,fips,cases_apr10,deaths_apr10
0,10000000,ito incorporated,431 isis ave,inglewood,ca,90301,722310,subchapter s corporation,500,2020-04-09,...,non-unique,los angeles,18670084713,103484958447,88778057202,-14706901245,2020-04-10,6037,8430,241
1,122194,sweet_&_chilli_llc,440 hindry ave unit h,inglewood,ca,90301,722511,limited liability company(llc),7,2020-06-16,...,non-unique,los angeles,18670084713,103484958447,88778057202,-14706901245,2020-04-10,6037,8430,241


In [58]:
for i in ['2020-01-01', '2020-02-01', '2020-03-01','2020-04-01', '2020-05-01','2020-06-01','2020-07-01', '2020-08-01','2020-09-01','2020-10-01']:
    employment_new = employment[employment['date']==i]
    employment_new['unemployment_rate_'+i] = employment_new['unemployment_rate']
    employment_new = employment_new.drop('unemployment_rate', axis=1)

    final_merged['unemployment_rate_'+i] = final_merged.merge(employment_new, on='county')['unemployment_rate_'+i]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [22]:
final_merged.columns

Index(['ppp_loan_amount', 'business_name', 'address', 'city', 'state',
       'zipcode', 'naics_code', 'business_type', 'jobs_reported',
       'date_approved', 'lender', 'cd', 'industry_title', 'two_digit_code',
       'general_industry_title', 'thousands_of_jobs_2019', 'zipcode_type',
       'county', 'county_total_loan_amount', 'q1', 'q2', 'q_dif', 'date',
       'fips', 'cases_apr10', 'deaths_apr10', 'unemployment_rate_2020-01-01',
       'unemployment_rate_2020-02-01', 'unemployment_rate_2020-03-01',
       'unemployment_rate_2020-04-01', 'unemployment_rate_2020-05-01',
       'unemployment_rate_2020-06-01', 'unemployment_rate_2020-07-01',
       'unemployment_rate_2020-08-01', 'unemployment_rate_2020-09-01',
       'unemployment_rate_2020-10-01'],
      dtype='object')

In [23]:
final_merged.to_csv('ayesha_merged_final.csv', index=False)

In [59]:
final_merged2=final_merged.groupby('general_industry_title').mean().reset_index()

In [60]:
melted_final=final_merged2.melt(id_vars=['general_industry_title'], value_vars=['unemployment_rate_2020-01-01', 'unemployment_rate_2020-02-01','unemployment_rate_2020-03-01','unemployment_rate_2020-04-01', 'unemployment_rate_2020-05-01','unemployment_rate_2020-06-01','unemployment_rate_2020-07-01', 'unemployment_rate_2020-08-01','unemployment_rate_2020-09-01','unemployment_rate_2020-10-01'], var_name='rate_date')
                               
                               
                               

In [61]:
melted_final['rate_date']=melted_final['rate_date'].str.strip('unemployment_rate_')


In [62]:
#number of employees per industry in thousands
job_counts=final_merged.groupby(['general_industry_title'])['thousands_of_jobs_2019'].mean()
#total loan amount per industry
loan_industry=final_merged.groupby('general_industry_title')['ppp_loan_amount'].sum()
#counts of each industry in dataset
industry_counts=final_merged['general_industry_title'].value_counts()

In [63]:
#function to normalize data so we can graph multiple charts w/different y-axis in one 
def norm(df):
    return (df-df.min())/(df.max()-df.min())

In [9]:
df = pd.DataFrame({'Employees per Industry': norm(job_counts),
                   'Loan Amount per Industry': norm(loan_industry),
                  'Industry Proportion in Dataset':norm(industry_counts)}, index=norm(industry_counts).index).sort_values(by='Employees per Industry', ascending=False)

In [64]:
df = pd.DataFrame({'Employees per Industry': norm(job_counts),
                   'Loan Amount per Industry': loan_industry,
                  'Industry Proportion in Dataset':norm(industry_counts)}, index=norm(industry_counts).index).sort_values(by='Employees per Industry', ascending=False)

In [67]:
df.to_csv('ayesha_industry_ppp4.csv')

In [70]:
bx = pd.read_csv('ayesha_industry_ppp4.csv', sep=',', index_col=0)
bx

,Employees per Industry,Loan Amount per Industry,Industry Proportion in Dataset
public administration,1.000000,180739019,0.007024
professional and business services,0.941537,11964674577,1.000000
leisure and hospitality,0.726731,6773907863,0.529912
retail trade,0.684484,4473485954,0.435621
manufacturing,0.557306,6292294485,0.258001
financial activities,0.371689,3535728752,0.425485
construction,0.314836,7170003166,0.369305
other services,0.279540,3360922471,0.496832
wholesale trade,0.242792,3662802155,0.209001
transportation and warehousing,0.229856,1760111407,0.179309


In [30]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=loan_industry.index.tolist(),
        y=loan_industry.tolist(),
        name='Loan Amount per Industry',
        marker=dict(
        color='rgb(102, 166, 30)'
    )
    ),secondary_y=True
)

fig.add_trace(go.Bar(
    y=df['Employees per Industry'].round(3).tolist(),
    x=df.index.tolist(),
    name='Employees per Industry',
    #orientation='h',
    marker=dict(
        color='rgb(228, 26, 28)'
    )
))
# fig.add_trace(go.Bar(
#     y=df.index.tolist(),
#     x=df['Loan Amount per Industry'].round(3).tolist(),
#     name='Loan Amount per Industry',
#     #orientation='h',
#     marker=dict(
#         color='rgb(55, 126, 184)'
#         #line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
#     )
# ))
fig.add_trace(go.Bar(
    y=df['Industry Proportion in Dataset'].round(3).tolist(),
    x=df.index.tolist(),
    name='Industry Proportion in PPP Loan Dataset',
    #orientation='h',
    marker=dict(
        color='rgb(55, 126, 184)'
        #line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
    )
))
fig.update_xaxes(title_text="Industry")
fig.update_yaxes(title_text="Proportion Relative to All Industries")
fig.update_yaxes(title_text="Total Loan Amount", secondary_y=True)
fig.update_layout(title='Industry & PPP Loan Data')
fig.show()

ValueError: Invalid property specified for object of type plotly.graph_objs.Bar: 'secondary'

Did you mean "xcalendar"?

    Valid properties:
        alignmentgroup
            Set several traces linked to the same position axis or
            matching axes to the same alignmentgroup. This controls
            whether bars compute their positional range dependently
            or independently.
        base
            Sets where the bar base is drawn (in position axis
            units). In "stack" or "relative" barmode, traces that
            set "base" will be excluded and drawn in "overlay" mode
            instead.
        basesrc
            Sets the source reference on Chart Studio Cloud for
            base .
        cliponaxis
            Determines whether the text nodes are clipped about the
            subplot axes. To show the text nodes above axis lines
            and tick labels, make sure to set `xaxis.layer` and
            `yaxis.layer` to *below traces*.
        c

In [11]:
melted_final['rate_date']=pd.to_datetime(melted_final['rate_date'])
melted_final['value']=melted_final['value'].round(3)

In [12]:
#mask = upd_df.continent.isin(continents)
fig = px.line(melted_final, 
    x="rate_date", y="value", color='general_industry_title')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text="Unemployment Rate")
fig.update_layout(title='Unemployment Rate per Industry Across 2020')
fig.show()

In [156]:
# from plotly.subplots import make_subplots
# import plotly.graph_objects as go

# fig = make_subplots(rows=1, cols=4, shared_xaxes=False, shared_yaxes=True,subplot_titles=("Jan 2020","Apr 2020", "July 2020", "Oct 2020"))

# fig.add_trace(go.Bar(x=jan_rate.index.tolist(), y=jan_rate['value'],
#                     marker=dict(color=jan_rate['value'], coloraxis="coloraxis")),
#               1, 1)

# fig.add_trace(go.Bar(x=apr_rate.index.tolist(), y=apr_rate['value'],
#                     marker=dict(color=apr_rate['value'], coloraxis="coloraxis")),
#               1, 2)
# fig.add_trace(go.Bar(x=july_rate.index.tolist(), y=july_rate['value'],
#                     marker=dict(color=july_rate['value'], coloraxis="coloraxis")),
#               1, 3)

# fig.add_trace(go.Bar(x=oct_rate.index.tolist(), y=oct_rate['value'],
#                     marker=dict(color=oct_rate['value'], coloraxis="coloraxis")),
#               1, 4)
# fig.update_xaxes(tickangle =40)
# #fig.update_yaxes(title_text="Unemployment Rate")
# fig.update_layout(title='Unemployment Rate per Industry Across 2020',coloraxis=dict(colorscale='Bluered_r'), showlegend=False)
# fig.show()
